# Coordinates retrieving of the closest image in the database - Inference

Workflow:

* encoding images using transfert learning from pretrained ResNet model on a geographic zones classification task
* binary hash images of training images in 512 feature space
* retrieving the closest image to the one to identify in that feature space using Locality Sensitive Hashing for fast approximate nearest neighbor

### Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import multiprocessing
from tqdm import notebook
import ast
import sys
import itertools
import os
from skimage import transform
import cv2 as cv

import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils

### Define path variables

In [2]:
###Define path to test data
path_test_CSV = "./data/piom_new_test_10k.csv"
path_test_image_folder = './data/piom_new_test_10k/'

###Define path to models
path_CNN = './models/gpu_model_resnet18.pth'
path_LSH = './models/lsh.pkl'

###Define path du submission CSV
path_submission_CSV = "submission.csv"

### Util functions

In [3]:
class MapDataset(Dataset):
    def __init__(self, coordinateDf, root_dir, transform=None):
        """
        Args:
            coordinateDf (pd.DataFrame): DataFrame with image id and geographic coordinates.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.map_coordinates = coordinateDf
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.map_coordinates)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir,
                                self.map_coordinates.iloc[idx, 0] + '.png')
        image = cv.imread(img_name)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image)
        coordinates_crnr = np.array([self.map_coordinates['llcrnrlon'].iloc[idx], 
                            self.map_coordinates['llcrnrlat'].iloc[idx],
                            self.map_coordinates['urcrnrlon'].iloc[idx], 
                            self.map_coordinates['urcrnrlat'].iloc[idx]]).astype('float')
        sample = {'image': image,
                  'Geo_zone': self.map_coordinates['Geo_zone'].iloc[idx],
                  'image_path': img_name,
                  'coordinates_crnr': coordinates_crnr}
        return sample

class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size 
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        # image = transform.resize(image, (new_h, new_w), mode='reflect', anti_aliasing=True)
        image = cv.resize(image, (new_h, new_w))
        return image

class SquareRescale(object):
    """Rescale the image in a sample to a given size."""

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        image = cv.resize(image, (self.output_size, self.output_size))
        # image = transform.resize(image, (self.output_size, self.output_size), mode='reflect', anti_aliasing=True)
        return image


class CenterCrop(object):
    """Crop the image in a sample centered to the middle of the image."""

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, image):
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        image = image[((h - new_h)//2): ((h - new_h)//2) + new_h,
                      ((w - new_w)//2): ((w - new_w)//2) + new_w]
        return image


class Normalize(object):
    """Normalize the image."""

    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def __call__(self, image):
        image = cv.normalize(image, None, alpha=self.alpha, beta=self.beta, 
                    norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
        return image


class ToTensor(object):
    """Convert ndarrays to torch tensors."""

    def __call__(self, image):
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return torch.from_numpy(image)

In [4]:
class CoordinatesFromLSH():
    """Retrieve the coordinates from """
    def __init__( self, lsh, features ):
        self.lsh = lsh
        self.features = features
    
    def get_similar_item_image_coordinates(self, i_features):
        try:
            response = self.lsh.query(self.features[i_features].flatten(), 
                            num_results=1, distance_func='hamming')
            return ast.literal_eval(response[0][0][1])
        except IndexError as error:
            print('Coordinate not found index: {}'.format(i_features))
            return [0, 0, 0, 0]
    
    def get_coordinates_multiprocessing(self):
        paramlist = list(range(len(self.features)))
        #Generate processes equal to the number of cores
        print('Getting coordinates...')
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        Coordinates = list(notebook.tqdm(pool.map_async(
            self.get_similar_item_image_coordinates, paramlist).get(), total=len(paramlist)))
        pool.close()
        pool.join()
        return Coordinates

In [5]:
def main_inference(path_test_CSV, path_test_image_folder, path_CNN, path_LSH, path_submission_CSV, test_batch_size):
    
    startTime = time.time()
    
    ###======== Load CNN model ============
    ###create model dropping last FC layer
    model = models.resnet18(progress=True)
    model = nn.Sequential(*list(model.children())[:-1])
    ###model on GPU if GPU is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    ###load previously trained model parameters
    model.load_state_dict(torch.load(path_CNN, map_location=device))
    print("Model loaded from {}".format(path_CNN))
    # print(model)
    
    ###======== Load LSH model ============
    lsh = pickle.load(open(path_LSH,'rb'))
    print("LSH loaded from {}".format(path_LSH))
    
    ###======== Test set dataloader ============
    testDF = pd.read_csv(path_test_CSV)
    testDF.drop(columns=["Unnamed: 0"],inplace=True)
    testDF['Geo_zone'] = np.nan
    test_map_data = MapDataset(testDF, path_test_image_folder, transform= transforms.Compose([
                                                       Rescale(225),
                                                       CenterCrop((224,224)),
                                                       Normalize(alpha=0., beta=1.),
                                                       ToTensor(),
                                                   ]))
    test_loader = torch.utils.data.DataLoader(test_map_data,
                                              batch_size=test_batch_size,
                                              shuffle=False,
                                              num_workers=multiprocessing.cpu_count(),
                                              drop_last=False,
                                              pin_memory=True)
    print('Test data loader created, number of cores: {}'.format(multiprocessing.cpu_count()))

    ###======= Test set forward pass========
    total_step = len(test_loader)
    test_features = []
    for i_batch, sample_batch in enumerate(test_loader):
        with torch.no_grad():
            test_X = sample_batch['image'].float().to(device)
            outputs = model(test_X)
            test_features.extend(outputs.detach().cpu().numpy())
        print('Batch: [{}/{}]'.format(i_batch+1, total_step), end="\r")
    print('Test features created!')
    
    ###========= Retrieve coordinates from LSH domain ============

    cooFromLSH = CoordinatesFromLSH(lsh, test_features)
    Coordinates = cooFromLSH.get_coordinates_multiprocessing()
    print('Coordinates retrieved!')
    ###copy coordinates to test dataframe
    # coordinates_transposed = zip(Coordinates)
    testDF[["llcrnrlon", "llcrnrlat", "urcrnrlon", "urcrnrlat"]] = pd.DataFrame(Coordinates)
    print(testDF.head())
    print("Test set - finding coordinates, time: {}h {}min {}s".format((time.time()-startTime)//3600, 
                                                                      ((time.time()-startTime)%3600)//60,
                                                                      (time.time()-startTime)%60))
    testDF[["id","llcrnrlon","llcrnrlat","urcrnrlon","urcrnrlat"]].to_csv(path_submission_CSV)
    print('CSV file ready to submit, saved: {}'.format(path_submission_CSV))

### Inference

In [1]:
if __name__ == '__main__':
    test_batch_size=50
    main_inference(path_test_CSV, path_test_image_folder, path_CNN, path_LSH, path_submission_CSV, test_batch_size)